# Ejercicio 2: Top 10 arrival airports in 2013

# Team Science t-ish

## Importamos librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bz2

In [2]:
#Empleamos el fichero 'bookings.csv.bz2'

In [ ]:
df_bookings=pd.read_csv('bookings.csv.bz2', sep='^', nrows=10) #Comprobamos las 10 primeras lineas

In [ ]:
df_bookings.head(2)

## OPCION 1: Abrirmos el fichero original por partes empleando la función "chunk"

In [ ]:
df_acum_1 = pd.DataFrame()


In [18]:
df_acum_2 = pd.DataFrame()

In [20]:
%%time

chunksize = 999999

reader = pd.read_csv('bookings.csv.bz2', 
                     compression = "bz2", 
                     sep = "^", 
                     memory_map = True, 
                     error_bad_lines = False,
                     usecols=["arr_port","pax"],
                     chunksize=chunksize
                    )    

CPU times: user 23.7 ms, sys: 0 ns, total: 23.7 ms
Wall time: 26.6 ms


In [11]:
%%time
for df in reader:      
    curr = df.groupby(by="arr_port").sum()
    if df_acum_1.empty:
        df_acum_1 = curr
    else:
        df_acum_1 = pd.concat([df_acum_1, curr])

CPU times: user 1min 47s, sys: 414 ms, total: 1min 48s
Wall time: 1min 48s


In [21]:
%%time
for df_chunk in reader:
    df_chunk=df_chunk.drop_duplicates()  #quitamos posibles duplicados
    df_chunk['arr_port'] = df_chunk['arr_port'].str.strip().str.upper()
    curr = df_chunk.groupby(by="arr_port").sum()
    if df_acum_2.empty:
        df_acum_2 = curr
    else:
        df_acum_2 = pd.concat([df_acum_2, curr])

CPU times: user 1min 46s, sys: 421 ms, total: 1min 46s
Wall time: 1min 47s


In [29]:
result = df_acum_1.groupby(by="arr_port").sum().sort_values(by="pax", ascending=False)[0:10]
result = result.astype({"pax": int})
result

,pax
arr_port,
LHR,88809
MCO,70930
LAX,70530
LAS,69630
JFK,66270
CDG,64490
BKK,59460
MIA,58150
SFO,58000


In [26]:
result = df_acum_2.groupby(by="arr_port").sum().sort_values(by="pax", ascending=False)[0:10]
result = result.astype({"pax": int})
result

,pax
arr_port,
LIS,2700
EWR,2380
CUN,1930
CDG,1800
JNB,1290
VIE,1240
LGW,1140
BLQ,1140
ATH,1060


## OPCION 2: Usamos la función "usecols" para cargar el fichero de una vez
### Quedándonos solo con las dos columneas que necesitamos

In [30]:
%%time
#LEEMOS SOLO LAS COLUMNAS arr_port y pax mediante la función rad_csv y el argumento "usecols"

df_bookings_2_colm = pd.read_csv("bookings.csv.bz2",
                                 compression= "bz2",
                                 sep = "^",
                                 error_bad_lines = False,
                                 memory_map = True,
                                 usecols = ["arr_port", "pax"])

CPU times: user 1min 53s, sys: 593 ms, total: 1min 53s
Wall time: 1min 54s


In [31]:
#Primer resultado sin filtrar nada
agrupacion_1=df_bookings_2_colm.groupby(by='arr_port').sum().sort_values(by='pax', ascending=False)[0:10]
agrupacion_1

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


### Limpiamos columnas, N#A, aseguramos tipo de campo y que no tengamos string raros

In [32]:
df_bookings_2_colm=df_bookings_2_colm.fillna({'pax':0})   #Rellenamos con 0 los campos  de pasajeros vacíos

In [33]:
df_bookings_2_colm['pax']=df_bookings_2_colm['pax'].astype(int)  #Quitamos decimales, pero me encuentro N#A

In [34]:
df_bookings_2_colm['arr_port']=df_bookings_2_colm['arr_port'].str.strip() #Retiramos los espacios en blanco

In [36]:
df_bookings_2_colm['arr_port']=df_bookings_2_colm['arr_port'].str.upper() #Todas Mayúsculas

In [37]:
#Agrupamos nº de pasajeros por aeropuertos
pasajeros_por_aeropuerto=df_bookings_2_colm.groupby('arr_port').sum()

In [38]:
#Agrupamos nº de pasajeros por aeropuertos
TOP_10=df_bookings_2_colm.groupby('arr_port').sum().sort_values(by = "pax", ascending= False)[0:10]
TOP_10

,pax
arr_port,
LHR,88809
MCO,70930
LAX,70530
LAS,69630
JFK,66270
CDG,64490
BKK,59460
MIA,58150
SFO,58000


### Debemos asegurar que el nombre del aeropuerto mantiene el formato de 3 Letras Mayúsculas

In [40]:
df_bookings_2_colm.head(2)

,arr_port,pax
0,LHR,-1
1,CLT,1


In [41]:
#Compruebo la longitud del campo 'arr_port' --> Tiene 8 de longitud.
a=df_bookings_2_colm['arr_port'].iloc[1]
len(a)
print(f"El aeropuerto es:xxxx{a}xxxx")

El aeropuerto es:xxxxCLTxxxx


In [42]:
df_bookings_2_colm.head(20)

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1
5,LGA,1
6,LGA,1
7,SIN,2
8,SIN,2
9,SIN,2


In [45]:
df_bookings_2_colm['arr_port_len']=df_bookings_2_colm['arr_port'].str.len()

In [46]:
df_bookings_2_colm['arr_port_len']==3

0           True
1           True
2           True
3           True
4           True
            ... 
10000005    True
10000006    True
10000007    True
10000008    True
10000009    True
Name: arr_port_len, Length: 10000010, dtype: bool

In [48]:
df_bookings_2_colm.groupby('arr_port_len').count()   #Compruebo si hay aeropuertos con más de 3 de longitud

,arr_port,pax
arr_port_len,,
2,1,1
3,10000009,10000009


In [49]:
df_bookings_2_colm[df_bookings_2_colm['arr_port_len']==2]

,arr_port,pax,arr_port_len
5000007,SG,0,2


# Cargamos csv con listado de aeropuertos mundiales

In [50]:
#https://github.com/opentraveldata/geobases/tree/public/GeoBases/DataSources/Airports/GeoNames

In [51]:
airport_names = pd.read_csv("airports_geonames_only_clean.csv",
                            sep = "^",
                            error_bad_lines = False,
                            memory_map = True,
                            names=('IATA','Nombre','IATA_2','IATA_Pais','Pais','Lat','Long'))
airport_names

,IATA,Nombre,IATA_2,IATA_Pais,Pais,Lat,Long
0,AUH,Abu Dhabi International Airport,AUH,AE,United Arab Emirates,24.432972,54.651138
1,AZI,Abu Dhabi Bateen Airport,AUH,AE,United Arab Emirates,24.428333,54.458084
2,AAN,Al Ain International Airport,AAN,AE,United Arab Emirates,24.261667,55.609167
3,DXB,Dubai International Airport,DXB,AE,United Arab Emirates,25.252778,55.364444
4,FJR,Fujairah,FJR,AE,United Arab Emirates,25.112225,56.323964
...,...,...,...,...,...,...,...
3554,OHD,Ohrid,OHD,NaN,NaN,41.179956,20.742325
3555,SKP,Skopje-Petrovec,SKP,NaN,NaN,41.961622,21.621381
3556,PSY,Stanley Airport,PSY,NaN,NaN,-51.685672,-57.777644
3557,TIH,Tikehau Atoll,TIH,NaN,NaN,-15.119442,-148.230457


In [52]:
airport_names['IATA'].str.len()

0       3
1       3
2       3
3       3
4       3
       ..
3554    3
3555    3
3556    3
3557    3
3558    3
Name: IATA, Length: 3559, dtype: int64

In [53]:
airport_names.groupby(airport_names['IATA'].str.len()).count()

,IATA,Nombre,IATA_2,IATA_Pais,Pais,Lat,Long
IATA,,,,,,,
3,3519,3519,3366,3506,3513,3519,3519
4,40,40,39,40,40,40,40


In [54]:
airport_names.sort_values(by = 'IATA', ascending= True) #Los códigos con 4 de longitud tienen un # delante

,IATA,Nombre,IATA_2,IATA_Pais,Pais,Lat,Long
2634,#AAA,Logan County Airport,AAA,US,United States,40.158654,-89.335095
2602,#ADU,Audubon County Airport,ADU,US,United States,41.701376,-94.920542
2315,#AGR,Rucks Dairy Airport,AGR,US,United States,27.701419,-81.448963
2855,#AIG,Langlade County Airport,AIG,US,United States,45.155523,-89.109001
2691,#ANE,Anoka County Airport,ANE,US,United States,45.144688,-93.211059
...,...,...,...,...,...,...,...
1451,ZVK,Savannakhet,ZVK,LA,Lao People's Democratic Republic,16.556594,104.759531
1506,ZWA,Andapa,ZWA,MG,Madagascar,-14.651667,49.620556
158,ZYL,Sylhet,ZYL,BD,Bangladesh,24.963242,91.866783
1557,ZZU,Mzuzu,ZZU,MW,Malawi,-11.444750,34.011776


In [55]:
airport_names['IATA'] = airport_names['IATA'].str.strip("#")  #Retiramos el # de los aeropuertos

In [56]:
airport_names.groupby(airport_names['IATA'].str.len()).count()

,IATA,Nombre,IATA_2,IATA_Pais,Pais,Lat,Long
IATA,,,,,,,
3,3559,3559,3405,3546,3553,3559,3559


In [57]:
listado_aeropuertos=airport_names[['IATA','Nombre']]

In [58]:
listado_aeropuertos.sort_values('IATA', ascending=True)

,IATA,Nombre
2253,3M5,Moontown Airport
3407,6L4,Logan County Airport Logan; West Virginia; USA
1812,AAA,Anaa Aiport
2634,AAA,Logan County Airport
684,AAC,El Arish
...,...,...
1451,ZVK,Savannakhet
1506,ZWA,Andapa
158,ZYL,Sylhet
1557,ZZU,Mzuzu


In [59]:
#Debemos asegurarnos que no tengamos IATAS repetidos y en caso de tenerlos, ver que sucede.

In [60]:
listado_aeropuertos[listado_aeropuertos['IATA']=='AAA']

,IATA,Nombre
1812,AAA,Anaa Aiport
2634,AAA,Logan County Airport


In [61]:
listado_aeropuertos.loc[listado_aeropuertos['IATA']=='AAA', "Nombre"]

1812             Anaa Aiport
2634    Logan County Airport
Name: Nombre, dtype: object

In [64]:
listado_aeropuertos.drop_duplicates(subset=['IATA'])

,IATA,Nombre
0,AUH,Abu Dhabi International Airport
1,AZI,Abu Dhabi Bateen Airport
2,AAN,Al Ain International Airport
3,DXB,Dubai International Airport
4,FJR,Fujairah
...,...,...
3554,OHD,Ohrid
3555,SKP,Skopje-Petrovec
3556,PSY,Stanley Airport
3557,TIH,Tikehau Atoll


In [65]:
listado_aeropuertos.drop_duplicates(subset=['IATA']).groupby('IATA').count()==2

,Nombre
IATA,
3M5,False
6L4,False
AAA,False
AAC,False
AAE,False
...,...
ZVK,False
ZWA,False
ZYL,False


In [66]:
TOP_10

,pax
arr_port,
LHR,88809
MCO,70930
LAX,70530
LAS,69630
JFK,66270
CDG,64490
BKK,59460
MIA,58150
SFO,58000


In [67]:
TOP_10.rename(columns={'pax':'Pasajeros'}, inplace=True)
TOP_10

,Pasajeros
arr_port,
LHR,88809
MCO,70930
LAX,70530
LAS,69630
JFK,66270
CDG,64490
BKK,59460
MIA,58150
SFO,58000


In [68]:
TOP_10['IATA']=TOP_10.index
TOP_10

,Pasajeros,IATA
arr_port,,
LHR,88809,LHR
MCO,70930,MCO
LAX,70530,LAX
LAS,69630,LAS
JFK,66270,JFK
CDG,64490,CDG
BKK,59460,BKK
MIA,58150,MIA
SFO,58000,SFO


In [69]:
TOP_10_FINAL = pd.merge(TOP_10, listado_aeropuertos, on="IATA", how='left')
TOP_10_FINAL

,Pasajeros,IATA,Nombre
0,88809,LHR,London Heathrow Airport
1,70930,MCO,Orlando International Airport
2,70530,LAX,Los Angeles International Airport
3,69630,LAS,McCarran International Airport
4,66270,JFK,John F Kennedy International Airport
5,64490,CDG,Paris - Charles-de-Gaulle
6,59460,BKK,Suvarnabhumi
7,58150,MIA,Miami International Airport
8,58000,SFO,San Francisco International Airport
9,55590,DXB,Dubai International Airport


In [70]:
TOP_10_FINAL.to_csv("Ejercicio_2_Resuelto.csv")